In [3]:
import csv
import os
from scenedetect import detect, ContentDetector, split_video_ffmpeg

# Set the base name
base_name = "4 Girls Beaten Shot Stabbed by Tough Man_1080pFHR"

# Construct file and directory names
video_file = f'{base_name}.mp4'
output_directory = f'{base_name}_split_videos'
csv_filename = os.path.join(output_directory, f'{base_name}_scene_detect.csv')

os.makedirs(output_directory, exist_ok=True)

def get_scene_info(scene):
    start_frame, start_timecode = int(scene[0].get_frames()), scene[0].get_timecode()
    end_frame, end_timecode = int(scene[1].get_frames()), scene[1].get_timecode()

    # Calculate lengths
    length_frames = end_frame - start_frame
    length_timecode_frames = int(scene[1].get_frames()) - int(scene[0].get_frames())
    length_timecode_seconds = scene[1].get_seconds() - scene[0].get_seconds()
    length_seconds = scene[1].get_seconds() - scene[0].get_seconds()

    return start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode_frames, scene[0].get_seconds(), scene[1].get_seconds(), length_timecode_seconds, length_seconds

with open(csv_filename, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Video Name', 'Scene Number', 'Start Frame', 'End Frame', 'Length (frames)', 'Start Timecode', 'End Timecode', 'Length (timecode)', 'Start Time (seconds)', 'End Time (seconds)', 'Length (seconds)'])

    for i, scene in enumerate(detect(video_file, ContentDetector())):
        split_video_name = f'{base_name}-Scene-{i + 1:03d}.mp4'
        print(f'Splitting scene {i + 1}: {split_video_name}')
        split_video_ffmpeg(video_file, [scene], output_file_template=os.path.join(output_directory, split_video_name), show_progress=True)

        start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode_frames, start_seconds, end_seconds, length_timecode_seconds, length_seconds = get_scene_info(scene)
        csv_writer.writerow([split_video_name, i + 1, start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode_frames, start_seconds, end_seconds, length_seconds])

print(f'CSV file "{csv_filename}" has been created.')
print(f'Split videos and CSV are saved in the "{output_directory}" directory.')


Splitting scene 1: 4 Girls Beaten Shot Stabbed by Tough Man_1080pFHR-Scene-001.mp4


100%|██████████| 688/688 [00:05<00:00, 127.07frame/s]


Splitting scene 2: 4 Girls Beaten Shot Stabbed by Tough Man_1080pFHR-Scene-002.mp4


100%|██████████| 176/176 [00:01<00:00, 108.48frame/s]


Splitting scene 3: 4 Girls Beaten Shot Stabbed by Tough Man_1080pFHR-Scene-003.mp4


...

Splitting scene 82: 4 Girls Beaten Shot Stabbed by Tough Man_1080pFHR-Scene-082.mp4


100%|██████████| 223/223 [00:02<00:00, 80.93frame/s]


Splitting scene 83: 4 Girls Beaten Shot Stabbed by Tough Man_1080pFHR-Scene-083.mp4


100%|██████████| 276/276 [00:02<00:00, 115.54frame/s]

CSV file "4 Girls Beaten Shot Stabbed by Tough Man_1080pFHR_split_videos\4 Girls Beaten Shot Stabbed by Tough Man_1080pFHR_scene_detect.csv" has been created.
Split videos and CSV are saved in the "4 Girls Beaten Shot Stabbed by Tough Man_1080pFHR_split_videos" directory.
